In [14]:
from autumn.tools.project import get_project
from matplotlib import pyplot
from autumn.tools.plots.utils import REF_DATE
import datetime
import pandas as pd
import os
import json
from autumn.settings.folders import PROJECTS_PATH

In [3]:
project = get_project("hierarchical_sir", "multi")

In [4]:
update_params = {
    "beta.AUS": .1,
    "beta.ITA": .2,
    "gamma": .1
}
params = project.param_set.baseline.update(update_params, calibration_format=True)

In [5]:
model = project.run_baseline_model(params)
derived_df = model.get_derived_outputs_df()

In [6]:
data = derived_df[::30]

In [12]:
countries = ["AUS", "ITA"]

timeseries = {}
for country in countries:
    times=data.index
    # plt.plot(times, values)

    times = [(t - pd.Timestamp(REF_DATE)).total_seconds() / 3600 / 24 for t in times]
    values = [int(v) for v in data[f"incidence_{country}"]]
    timeseries[f"incidence_{country}"] = {
        "output_key": f"incidence_{country}",
        "title": f"Incidence of {country}",
        "times": times,
        "values": values,
        "quantiles": [.025, 0.25, 0.5, 0.75, 0.975]
    }


In [15]:
targets_path = os.path.join(PROJECTS_PATH, "hierarchical_sir", "multi", "timeseries.json")
 
# Writing to sample.json
with open(targets_path, "w") as outfile:
    json.dump(timeseries, outfile, indent=2)

In [ ]:
for output_type in ["incidence_AUS", "incidence_ITA"]:
    outputs = [o for o in derived_df.columns if o.startswith(output_type)]
    
    fig = pyplot.figure(figsize=(12, 8))
    pyplot.style.use("ggplot")
    axis = fig.add_subplot()
    for output in outputs:
        axis = derived_df[output].plot()
        
    axis.set_title(output_type)
    # axis.set_xlim(("Jan 2020", "May 2020"))
    axis.legend()
    